# Visualize high-redshift candidates in COSMOS using Firefly

Notebook developed by D. Shupe and T. Goldina

The purpose of this notebook is to visualize candidates from Figure 4 of [Bowler et al 2012](https://academic.oup.com/mnras/article/426/4/2772/1012488#17888103) that are the best high-z candidates, and to assess the astrometry by overlaying Gaia sources.

Let's gather the coordinates from Table 2 of Bowler et al. using astroquery and NED. Note that Vizier does not have the tables from this paper.

In [ ]:
from astroquery.ned import Ned

In [ ]:
candidates = Ned.query_refcode('2012MNRAS.426.2772B')

In [ ]:
candidates

We'll use Firefly for retrieving images and visualizing them, and for overlaying Gaia stars.
The goal is to assess by eye the astrometry.

In [ ]:
import firefly_client

We'll verify which Firefly server we are using.

In [ ]:
import os

In [ ]:
os.environ.get('FIREFLY_URL')

Verify we are set up to use Slate

In [ ]:
os.environ.get('FIREFLY_HTML')

The next cell will open up a Firefly tab in Jupyterlab.

In [ ]:
fc = firefly_client.FireflyClient.make_lab_client()

The surveys that Firefly can access directly are available on Github at https://github.com/Caltech-IPAC/firefly/blob/dev/src/firefly/java/edu/caltech/ipac/firefly/resources/irsa-image-master-table.csv

Let's read those into an Astropy table.

In [ ]:
from astropy.table import Table

In [ ]:
surveys = Table.read('https://raw.githubusercontent.com/Caltech-IPAC/firefly/dev/src/firefly/' +
                     'java/edu/caltech/ipac/firefly/resources/irsa-image-master-table.csv',
                     format='csv')

In [ ]:
cosmos_surveys = surveys[surveys['missionId'] == 'COSMOS']
cosmos_surveys

Table 2 and Figure 4 of Bowler et al 2012 shows photometry in z', Y, J, H, K_s, and Spitzer 3.6um and 4.5um. We want the surveys that correspond to these. 

In [ ]:
import numpy as np
np.unique(cosmos_surveys['wavebandId'].data)

In [ ]:
cosmos_surveys[cosmos_surveys['wavebandId'] == 'z+']

## Restart for another candidate (index) here

Let's pick a candidate galaxy from our table.

In [ ]:
candidate_index = 3
tbl_index = candidates['No.'] == 3

In [ ]:
fc.reinit_viewer()

In [ ]:
from astropy.coordinates import SkyCoord
import astropy.units as u

In [ ]:
c = SkyCoord(ra=candidates['RA'][tbl_index].data[0]*u.deg,
             dec=candidates['DEC'][tbl_index].data[0]*u.deg)

In [ ]:
target = '{};{};EQ_J2000'.format(c.ra.deg, c.dec.deg)

In [ ]:
target

Use the Firefly search processor for Gaia DR2.

These are Gator queries. Let's show how we find the information.

In [ ]:
irsacats = Table.read('https://irsa.ipac.caltech.edu/cgi-bin/Gator/nph-scan?mode=ascii', format='ipac')
projects = np.unique(irsacats['projectshort'].data)
projects

In [ ]:
irsacats[irsacats['projectshort'] == 'Gaia']

In [ ]:
irsacats[irsacats['cntr'] == 638]['catname'][0]

In [ ]:
fc.show_table(tbl_id='gaiaTbl', title='Gaia DR2 catalog',
             target_search_info=dict(catalogProject='Gaia',
                                     catalog='gaia_dr2_source',
                                     position=target,
                                     SearchMethod='Box',
                                     size=400),
             options=dict(removable=True, showUnits=False, showFilters=True))

The next cell lists some information on what we need to provide to Firefly's image search processors

In [ ]:
fc.show_fits(plot_id='z+', Title='z+', Type='SERVICE', Service='ATLAS',
            SurveyKey='cosmos.cosmos_subaru', SurveyKeyBand='z+',
            WorldPt=target, SizeInDeg=400/3600);

In [ ]:
fc.show_fits(plot_id='Y', Title='Y', Type='SERVICE', Service='ATLAS',
            SurveyKey='cosmos.cosmos_ultravista', SurveyKeyBand='Y',
            WorldPt=target, SizeInDeg=400/3600);

In [ ]:
fc.show_fits(plot_id='H', Title='H', Type='SERVICE', Service='ATLAS',
            SurveyKey='cosmos.cosmos_ultravista', SurveyKeyBand='H',
            WorldPt=target, SizeInDeg=400/3600);

In [ ]:
fc.show_fits(plot_id='K', Title='K', Type='SERVICE', Service='ATLAS',
            SurveyKey='cosmos.cosmos_ultravista', SurveyKeyBand='K',
            WorldPt=target, SizeInDeg=400/3600);

In [ ]:
fc.show_fits(plot_id='IRAC1', Title='3.6um', Type='SERVICE', Service='ATLAS',
            SurveyKey='cosmos.cosmos_irac', SurveyKeyBand='IRAC1',
            WorldPt=target, SizeInDeg=400/3600);

In [ ]:
fc.show_fits(plot_id='IRAC2', Title='4.5um', Type='SERVICE', Service='ATLAS',
            SurveyKey='cosmos.cosmos_irac', SurveyKeyBand='IRAC1',
            WorldPt=target, SizeInDeg=400/3600);

In the Firefly tab, lock all the displays and turn on WCS match. Then, execute the following cells, not too quickly, to allow the actions to take effect for each one.

In [ ]:
fc.set_stretch('z+', 'zscale', 'asinh');

In [ ]:
fc.set_zoom('z+', 0.25);

In [ ]:
fc.dispatch(action_type='ImagePlotCntlr.ColorChange',
            payload=dict(plotId='z+',
            cbarId=1));